<a href="https://colab.research.google.com/github/FeelOneE/Attention-is-all-you-need-pwlee/blob/main/Attention_is_all_you_need.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/ArazShilabin/english-to-german-translator-using-transformers-from-scratch/blob/main/en_de_translator.ipynb

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%pwd

'/content'

In [5]:
%cd /content/drive/MyDrive/스터디/기계이해

/content/drive/MyDrive/스터디/기계이해


# (준비) 데이터셋 및 불용어(non breaking prefix) 데이터 다운로드

In [ ]:
%cd data
!wget http://www.statmt.org/europarl/v7/de-en.tgz
!tar -xvf de-en.tgz
%cd ..

--2021-12-17 10:38:24--  http://www.statmt.org/europarl/v7/de-en.tgz
Resolving www.statmt.org (www.statmt.org)... 129.215.197.184
Connecting to www.statmt.org (www.statmt.org)|129.215.197.184|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.statmt.org/europarl/v7/de-en.tgz [following]
--2021-12-17 10:38:24--  https://www.statmt.org/europarl/v7/de-en.tgz
Connecting to www.statmt.org (www.statmt.org)|129.215.197.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 197785698 (189M) [application/x-gzip]
Saving to: ‘de-en.tgz’

de-en.tgz           100%[===================>] 188.62M  1.52MB/s    in 2m 4s   

2021-12-17 10:40:28 (1.52 MB/s) - ‘de-en.tgz’ saved [197785698/197785698]

europarl-v7.de-en.de
europarl-v7.de-en.en


In [ ]:
%cd data
!wget https://raw.githubusercontent.com/moses-smt/mosesdecoder/master/scripts/share/nonbreaking_prefixes/nonbreaking_prefix.de
!wget https://raw.githubusercontent.com/moses-smt/mosesdecoder/master/scripts/share/nonbreaking_prefixes/nonbreaking_prefix.en
%cd ..

--2021-12-17 10:50:33--  https://raw.githubusercontent.com/moses-smt/mosesdecoder/master/scripts/share/nonbreaking_prefixes/nonbreaking_prefix.de
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1768 (1.7K) [text/plain]
Saving to: ‘nonbreaking_prefix.de’

nonbreaking_prefix. 100%[===================>]   1.73K  --.-KB/s    in 0.003s  

2021-12-17 10:50:33 (531 KB/s) - ‘nonbreaking_prefix.de’ saved [1768/1768]

--2021-12-17 10:50:33--  https://raw.githubusercontent.com/moses-smt/mosesdecoder/master/scripts/share/nonbreaking_prefixes/nonbreaking_prefix.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443

In [6]:
%cd data
%ls -al
%cd ..

/content/drive/MyDrive/스터디/기계이해/data
total 2298971
-rw------- 1 root root 296468668 Dec 22 07:40 cleaning.de-en.en
-rw------- 1 root root 622086459 Dec 22 07:40 cleaning.en-de.de
-rw------- 1 root root 622086459 Dec 22 06:08 cleaning.en-de.en
-rw------- 1 root root 197785698 May 16  2012 de-en.tgz
-rw------- 1 root root 328463491 Nov 21  2011 europarl-v7.de-en.de
-rw------- 1 root root 287250069 Nov 21  2011 europarl-v7.de-en.en
-rw------- 1 root root      1768 Dec 17 10:50 nonbreaking_prefix.de
-rw------- 1 root root      1339 Dec 17 10:50 nonbreaking_prefix.en
/content/drive/MyDrive/스터디/기계이해


# **step 1. 라이브러리 추가**

In [7]:
import numpy as np
import math
import re
import time

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds

# step 2. 데이터 전처리

In [8]:
with open("data/europarl-v7.de-en.en", mode='r', encoding="utf-8") as f:
    text_en = f.read()

with open("data/europarl-v7.de-en.de", mode='r', encoding="utf-8") as f:
    text_de = f.read()

print(text_en[:50])
print(text_de[:50])

Resumption of the session
I declare resumed the se
Wiederaufnahme der Sitzungsperiode
Ich erkläre die


In [9]:
with open("data/nonbreaking_prefix.en", mode="r", encoding="utf-8") as f:
  non_breaking_prefix_en = f.readlines()

tmp = []
for line in non_breaking_prefix_en:
  if line[0] == "#" or line[0] == "\n" :
    pass
  else:
    tmp.append(  (line.replace("\n","")).replace("#NUMERIC_ONLY#","").strip()  )
non_breaking_prefix_en = tmp
non_breaking_prefix_en[:5]

['A', 'B', 'C', 'D', 'E']

In [10]:
with open("data/nonbreaking_prefix.de", mode="r", encoding="utf-8") as f:
  non_breaking_prefix_de = f.readlines()

tmp = []
for line in non_breaking_prefix_de:
  if line[0] == "#" or line[0] == "\n" :
    pass
  else:
    tmp.append(  (line.replace("\n","")).replace("#NUMERIC_ONLY#","").strip()  )
non_breaking_prefix_de = tmp
non_breaking_prefix_de[:5]

['A', 'B', 'C', 'D', 'E']

In [ ]:
for prefix in non_breaking_prefix_en:
    text_en = text_en.replace(prefix, prefix + '###')
text_en = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".###", text_en)
text_en = re.sub(r"\.###", '', text_en)
text_en = re.sub(r" +", ' ', text_en)
text_en = text_en.replace('###', ' ')

text_en = text_en.split("\n")

for prefix in non_breaking_prefix_de:
    text_de = text_de.replace(prefix, prefix + '###')
text_de = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".###", text_de)
text_de = re.sub(r"\.###", '', text_de)
text_de = re.sub(r" +", ' ', text_de)
text_de = text_de.replace('###', ' ')

text_de = text_de.split("\n")

In [ ]:
print(text_en[:5])
print(text_de[:5])

['R esumption of the session', 'I  declare resumed the session of the E uropean P arliament adjourned on F riday 17 D ecember 1999, and I  would like once again to wish you a happ y new year in the hope that you enjoyed a pleasant festiv e period.', "A lthough, as you will hav e seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.", 'Y ou hav e requested a debate on this subject in the course of the next few days, during this part-session.', "I n the meantime, I  should like to observ e a minute' s silence, as a number of M embers hav e requested, on behalf of all the v ictims concerned, particularly those of the terrible storms, in the v arious countries of the E uropean U nion."]
['W i  e d e r a u f n a h m e  d e r  S i  t z u n g s p e r i  o d e ', 'I  c h  e r k l är e  d i  e  a m  F r e i  t a g , d e m  1 7 . D e z e m b e r  u n t e r b r o c h e n e  S i  t z u n 

In [ ]:
# 전처리한 데이터 저장
with open("data/cleaning.de-en.en","w") as f:
  f.write('\n'.join(text_en))

with open("data/cleaning.en-de.de","w") as f:
  f.write('\n'.join(text_de))

In [11]:
# 전처리한 데이터 불러오기
with open("data/cleaning.de-en.en","r+") as f:
  text_en = f.read()

with open("data/cleaning.en-de.de","r+") as f:
  text_de = f.read()

In [12]:
print(text_en[:50])
print(text_de[:500])

R esumption of the session
I  declare resumed the 
W i  e d e r a u f n a h m e  d e r  S i  t z u n g s p e r i  o d e 
I  c h  e r k l är e  d i  e  a m  F r e i  t a g , d e m  1 7 . D e z e m b e r  u n t e r b r o c h e n e  S i  t z u n g s p e r i  o d e  d e s  E u r o p äi  s c h e n  P a r l a m e n t s  f ür  w i  e d e r a u f g e n o m m e n , w ün s c h e  I  h n e n  n o c h m a l s  a l l e s  G u t e  z u m  J a h r e s w e c h s e l  u n d  h o f f e , d a ß S i  e  s c h ön e  F e r i  e n  h a t t e n .
W i  e  S i  e  f e s 


# stop 3. 토크나이저
> 토크나이저 :  각 텍스트에 대응하는 숫자들을 이해한다. 그러므로, 우리는 어떤 텍스트를 어떤식으로 분리해서, 분리된 텍스트를 특정한 숫자(id)에 대응시키고, 해당 id를 모델의 입력으로 넣어주는 모듈.
크게 3가지 방식이 사용된다 : word-based, character-based, subword-based.

>  subword-based : word-based와 character-based 사이의 방식이다. 자주 사용되는 단어들은 subword로 분리하지 않고, 자주 등장하지 않는 단어들은 의미있는 subword로 분리하는 방법이다. 예를 들어, dogs라는 단어는 자주 사용하는 dog와 ##s 로 분리할 수 있다. 영어에서 가장 좋은 성능을 보이고 있느느 tokenizer 이다.

> "##prefix' : BERT가 사용하는 subword tokenizer인 wordpiece tokenizer에서 사용하는 방식인데, 특정 token 앞에 ##가 붙는다면 해당 token은 독립적으로 사용할 수 있는 token이 아닌, 다른 단어를 구성하는 일부라는 의미이다.



In [ ]:
!pip install transformers

In [ ]:
%tensorflow_version 2.x
from transformers import AutoTokenizer

tokenizer_en = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer_de = AutoTokenizer.from_pretrained('GroNLP/bert-base-dutch-cased')

In [15]:
tokenizer_en.vocab_size, tokenizer_de.vocab_size

(30522, 30073)

In [ ]:
encoded_input = tokenizer_en(text_en, return_tensors='tf')
encoded_input

In [ ]:
encoded_output = tokenizer_de(text_de, return_tensors='tf')
encoded_output[:5]

In [ ]:
VOCAB_SIZE_EN = tokenizer_en.vocab_size + 2
VOCAB_SIZE_DE = tokenizer_de.vocab_size + 2

# we put start and end tokens as size-1 and size-2 which are the same as
# tokenizer_size and tokenizer_size+1 because the words are from [0 to ts-1]
# tokenize_en.encode(sentence) give a list then list + list + list appends them
inputs = [[VOCAB_SIZE_EN-2] + tokenizer_en.encode(sentence) + [VOCAB_SIZE_EN-1]
          for sentence in text_en]
outputs = [[VOCAB_SIZE_DE-2] + tokenizer_de.encode(sentence) + [VOCAB_SIZE_DE-1]
          for sentence in text_de]